<a href="https://colab.research.google.com/github/faizanshaikh0303/ExploringDaft/blob/main/ExploringDaft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas pyarrow daft duckdb



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 16.8 MB/s eta 0:00:00


In [10]:
import time
import pandas as pd
import daft
import pyarrow as pa
from daft import col

In [4]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
print("Loading dataset...")
df_pd = pd.read_parquet(url)
print(f"Dataset shape: {df_pd.shape}")
df_pd.head()

Loading dataset...
Dataset shape: (3066766, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
df_arrow = pa.Table.from_pandas(df_pd)
df_daft = daft.from_arrow(df_arrow)

In [6]:
print("\nQuery: Average fare amount per passenger count (fares > $10)\n")


Query: Average fare amount per passenger count (fares > $10)



In [24]:
start = time.time()
result_pd = (
    df_pd[df_pd["fare_amount"] > 10]
    .groupby("passenger_count")["fare_amount"]
    .mean()
    .reset_index()
)
t_pandas = time.time() - start
print(f"Pandas time: {t_pandas:.2f}s")

Pandas time: 0.30s


In [27]:
start = time.time()
result_daft = (
    df_daft.where(col("fare_amount") > 10)
    .groupby("passenger_count")
    .agg(col("fare_amount").mean().alias("avg_fare"))
)
result_daft = result_daft.to_pandas()
t_daft = time.time() - start
print(f"Daft time: {t_daft:.2f}s")


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Filter: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 GroupedAggregate: 00:00 

Daft time: 0.17s


In [28]:
print("\nResults:")
display(result_pd.head(10))
display(result_daft.head(10))

print(f"\n🏁 Performance Comparison: Daft = {t_daft:.2f}s | Pandas = {t_pandas:.2f}s")


Results:


,passenger_count,fare_amount
0,0.0,22.365433
1,1.0,24.097599
2,2.0,26.972824
3,3.0,26.233507
4,4.0,27.697271
5,5.0,23.695368
6,6.0,23.701842
7,7.0,68.166667
8,8.0,82.113077
9,9.0,90.000000


,passenger_count,avg_fare
0,7.0,68.166667
1,2.0,26.972824
2,6.0,23.701842
3,9.0,90.000000
4,1.0,24.097599
5,0.0,22.365433
6,8.0,82.113077
7,3.0,26.233507
8,5.0,23.695368
9,NaN,22.689362



🏁 Performance Comparison: Daft = 0.17s | Pandas = 0.30s


In [30]:
!pip install nbstripout
!nbstripout --install
!nbstripout ExploringDaft.ipynb


fatal: --local can only be used inside a git repository
Installation failed: not a git repository!
Could not strip 'ExploringDaft.ipynb': file not found
